In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
PREFER_CUDA = True

In [3]:
torch.cuda.is_available()

True

In [4]:
use_cuda = PREFER_CUDA and torch.cuda.is_available()

# args.seed
batch_size = 100
nb_epochs = 5

'''kwargs = {'num_workers': 1, 'pin_memory': True} if else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   
    batch_size=, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       
    batch_size=, shuffle=True, **kwargs)
'''

vocab_size = 250
n_words = 3
n_embed = 50
n_hidden = 200
n_output = vocab_size

learning_rate = 0.1
momentum = 0.9

In [5]:
embedding = nn.Embedding(vocab_size, 8)
# a batch of 2 samples of 4 indices each
input = Variable(torch.LongTensor([[11,20,4],[30,10,20],]))
print(embedding(input))

Variable containing:
(0 ,.,.) = 
 -0.7296  0.3700 -0.6237  0.3776 -1.9687 -0.1128 -0.0183 -0.2710
 -0.3206  0.6622 -0.7998 -0.0402 -0.3176  0.3279  0.7707  1.0483
 -0.4794  1.0229 -1.2639  0.6512 -1.9709 -0.8802 -0.0275 -1.4770

(1 ,.,.) = 
 -0.0267 -0.0223 -1.6432  1.0823  0.8049 -0.5796  1.1788 -0.5864
 -0.5742 -0.6410 -1.4154  1.2576  0.0382  1.8365 -0.4074  0.0912
 -0.3206  0.6622 -0.7998 -0.0402 -0.3176  0.3279  0.7707  1.0483
[torch.FloatTensor of size 2x3x8]



In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convert word into vectors
        # see documentation : http://pytorch.org/docs/nn.html#torch.nn.Embedding
        self.W2E = nn.Embedding(vocab_size, n_embed, padding_idx=None, max_norm=None, norm_type=2)
        self.E2H = nn.Linear(n_words*n_embed, n_hidden, bias=True)
        self.H2O = nn.Linear(n_hidden, n_output, bias=True)
        #self.conv2_drop = nn.Dropout2d()

    def forward(self, words):
        wvectors = self.W2E(words)
        # WARNING : wvectors may need a reshape operation ???
        h_activation = self.E2H(wvectors)
        h = F.sigmoid(h_activation)
        out_activation = self.E2H(h)
        #x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        #return F.log_softmax(x)
        return F.softmax(out_activation) # WARNING : or log_softmax ???

model = Net()
if use_cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss = F.cross_entropy(output, target, size_average=True)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, nb_epochs + 1):
    #train(epoch)
    #test(epoch)
    None
